In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from bidding.operations import create_bidding_app, setup_bidding_app, place_bid, cancel_bid, place_accept, close_bidding
from store.operations import StoringPool
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))

#close_bidding(client, 693, storeAppID, creator)

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


In [4]:
#n_pri_key, n_address = generate_rekeyed_algorand_keypair(client, bidder)
n_address = 'FOWXDCFQJKY3ISR7A5ZC2K5QEA5EYOJYCML55JWZVZYWWKQSUD6UFWIWKU'
set_rekeyed_address(bidder.get_address(), n_address, 1)

obj {'LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I': {'5TG3X5EGVIRUWLL72O4GRLDQKMLXWMKVMCVEFZRTNXQ65MAMW3UNUQ6DUI': 1}, 'Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM': {'67KQREK4UKL3BJFBPTTGGHF5STPU2QQ3KXLF7ZNBIKNHODJQ7K25OV7U5M': 1, 'E4U25BKMR3UNDB4K7HB3HYP6IIX2343MFM763EKCHFVWQBUY3RUCHYU53E': 1, 'NNA5OCYLEP2HAVDHB4OLKVB4GXGGUQD2JOF3SEUO3WLQUCLG56E2WJC3T4': 1, 'GJHC7JQ33S4PFBCBKAJSL5GL5SDTLDBFNJLB2ZLZF67BYIWIDEOOXQYZMY': 1, 'VXYVLO35ZAQMKW7BTE226QVVLDKMQHJJZRDBIYTK6T5OUK4ISZCSTSQH5Y': 1, 'DOMDTBBDCQCR7YSHHEOHQGRIU5SGAW3XIHDHP73VFSLRNRQVAXMDC4VZCY': 1, 'ORJTBC3UBKFVKALFIORVZNWSVOX23THDHEQMGQNSXQ2ABNSKH64OMYFEGQ': 1, 'ZTKLAEKCKCKTYV2TEOQNPQN45VG4ZRM6KKZHP7GC6YZQ4PDT2TE4SHQBIA': 1, 'GIU26NSBSMV62TTIT64O7V7YFLEYZEHZSYQYEKKLSF5QBYODXNWV2CGMF4': 1, 'VUINBMP75F446NEGXDY3BWAGXDIBWRSRFDNQ5SEVSRAS5SNELZ662YN7AY': 1, '6WUN7376ILJCYKNCWRLDCRIIAECD4SKCBXZWPTBNBWGBTX43EUOMGOKEFY': 1, '6IWEQJHKDP45O35COSVCBZRQAXKSVCQQGNEUNZJ7MU6RUGUTT4TH4ULBLA': 1}}


In [5]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 499


In [6]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [7]:
storeApp = StoringPool(client=client, creator=creator, token_id=asset_id)
# storeApp.create_app()
# storeAppID = storeApp.app_id

storeAppID = int(os.environ.get("STORE_APP_ID"))
storeApp.app_id = storeAppID
print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Store App ID: 727
Store App Address: 5YFHLDU7ELKNT3N25YKEXEHALZL6C3UEZLKZOSMICARY3ZBXDEPRKAKYWU


In [8]:

appID = create_bidding_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction U5CF34XSSS4W6JNVNLFHAWS3UG5BEUH4SO4KTQCEKPPZB7ZJ4HCQ confirmed in round 77881.
App ID: 1501
App Address: K6QNV5QRYXLNVNCK66CCRU3MYGEDKPVWZIFHWMMIAWEGP23U67DFKCEP6E


Alice is setting up and funding token auction...

In [9]:
setup_bidding_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_id=asset_id,
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Waiting for confirmation...
Transaction QR4OWWVHKYR2FQ5SDPH6SPQVTPIRT3SCQVFZK74WF3B55DZOTUOQ confirmed in round 77883.
Alice's algo balance:  20037617065184  algos


In [10]:
sleep(5)
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 100101000, 499: 0}


In [11]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

Carla wants to bid on token, her algo balance:  10019412902132  algos
Carla is placing bid for:  1000000  algos


In [12]:
bid_index = place_bid(client=client, app_id=appID, token_id=asset_id, bidder=bidder, bid_amount=10, bid_price=bidPrice, bid_index="")

print("Carla bid", asset_id)

store_app_id 727
new address: LGAL4UAIZO2NCSJEWCNS3UQBD37DVQQM4EZIZAOKDY6IUMN6Z7BEQ2YZAM
new private_key: IqCmfGKM1ORIKWuooGBGZY+tPeht/gMZtu/XYvk2aBJZgL5QCMu00UkksJst0gEe/jrCDOEyjIHKHjyKMb7Pwg==
new passphrase: affair once dinner shiver poverty decrease engage hen bench little permit wagon force author hungry wrist call such water coin version asset nation abandon embark
Waiting for confirmation...
Transaction 6KEKSGDA6GYP6LPKDBEGYHDOOOD7ZDI4D3KNG4GDJ33LBSJOFASA confirmed in round 77886.
Waiting for confirmation...
Transaction EBRWFBDIU7VUMREKZWERGJ6A3XV56VPUPDP6RFUO3ODZY3YFCMNQ confirmed in round 77888.
Waiting for confirmation...
Transaction 4JTKET537ZDC5EJ24NPIGD4ZR3K6NGTRYTQWLGV3FVGIK2APQVWQ confirmed in round 77890.
obj {'LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I': {'5TG3X5EGVIRUWLL72O4GRLDQKMLXWMKVMCVEFZRTNXQ65MAMW3UNUQ6DUI': 1, 'FOWXDCFQJKY3ISR7A5ZC2K5QEA5EYOJYCML55JWZVZYWWKQSUD6UFWIWKU': 1}, 'Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM': {'67KQRE

In [13]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

cancel_bid(client, appID, bidder, bid_index)

print("Carla canceled bid", asset_id)

Waiting for confirmation...
Transaction FEFA4DQUVY44Z2V5ZDEWL4XFM7K6GQ242V2AWNFTUFOUDWVJXXTA confirmed in round 77894.
Carla canceled bid 499


In [14]:
bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=10, bid_price=bidPrice, bid_index="")
print(bid_index)
bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=20, bid_price=bidPrice+1_000_000, bid_index="")
print(bid_index)
bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=15, bid_price=bidPrice+500_000, bid_index=bid_index)
print(bid_index)


store_app_id 727
local state of LGAL4UAIZO2NCSJEWCNS3UQBD37DVQQM4EZIZAOKDY6IUMN6Z7BEQ2YZAM : {b'TA': 0, b'TK_ID': 0, b'TP': 0}
Waiting for confirmation...
Transaction OUJTTT32L4VQCB3AOYL2AW5Y353WLZD7PJJSYXVNQSIJGZDT3WAA confirmed in round 77896.
LGAL4UAIZO2NCSJEWCNS3UQBD37DVQQM4EZIZAOKDY6IUMN6Z7BEQ2YZAM
store_app_id 727
local state of LGAL4UAIZO2NCSJEWCNS3UQBD37DVQQM4EZIZAOKDY6IUMN6Z7BEQ2YZAM : {b'TA': 10, b'TK_ID': 499, b'TP': 1000000}
new address: WBXOIFZ3T7NWHHWF74Z6AY7G4DZUEOIMXONLEV4QAGIDZ2Q3O44IIHQA3Q
new private_key: d5wbpdmThEWuI+x/OB2J7C8EVF2b0EM/jBdDnOkv07ywbuQXO5/bY57F/zPgY+bg80I5DLuasleQAZA86ht3OA==
new passphrase: moment hover spoon diesel ankle multiply buddy umbrella avocado insane duty yard analyst fence sustain spatial trick couch mesh service spring slogan track absent lobster
Waiting for confirmation...
Transaction 3PVI557P7BJCYPZ5FQOEZD2TJEP6IEKNSVDQYKZKUSVQULNFLWPA confirmed in round 77898.
Waiting for confirmation...
Transaction XBRF2JRK6MJP3HT52WYTVLYDU7TO5MX6D25

In [15]:
bidderAppLocalState = get_app_local_state(client, appID, bidder.get_address())
print(bidderAppLocalState)

{}


In [16]:
place_accept(client, creator, appID, seller, bidder.get_address(), bid_index)

False

Alice is closing out the bidding contract....

In [17]:
sleep(5)

close_bidding(client, appID, creator, [asset_id])

b'assets' [499]
b'accounts' ['4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64', 'UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU']
Waiting for confirmation...
Transaction IQ77VPHVRTKX3KAFOLHBKOCMIPZ4YAEL3NCB7C4AACFI6HKPDG3A confirmed in round 77909.


In [18]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 102599100, 499: 0}


In [19]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidPrice - 1_000
assert actualSellerBalances[asset_id] == 0

Carla's NFT balance: 31  for token ID:  499


AssertionError: 

In [ ]:
storeAppState = get_app_global_state(client, 727)
print(storeAppState)
print(encoding.encode_address(storeAppState[b"C"]))